## Default Setting

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from konlpy.tag import Okt
import sqlite3

        
# 🔹 제외할 영어 관사(articles) 목록
ARTICLES = {"a", "an", "the"}

# 🔹 한글 조사 목록 (단어 끝에 붙는 경우 의미 분석 방해 → 제거 대상)
PARTICLES = {
    "은",
    "는",
    "이",
    "가",
    "을",
    "를",
    "에",
    "에서",
    "와",
    "과",
    "도",
    "의",
    "한",
    "로",
    "으로",
    "하고",
    "및",
    "등",
    "까지",
    "부터",
    "만",
    "보다",
    "처럼",
    "같이",
    "께서",
}

## URL Parsing

In [7]:


def get_naver_economy_news_urls_from_list(pages=5):
    base_url = "https://news.naver.com/main/list.naver"
    headers = {"User-Agent": "Mozilla/5.0"}
    all_links = set()

    for page in range(1, pages + 1):
        params = {
            "mode": "LSD",
            "mid": "shm",
            "sid1": "101",  # 경제 뉴스 섹션
            "page": str(page)
        }
        response = requests.get(base_url, headers=headers, params=params)
        print(f"{page}페이지 응답 코드: {response.status_code}")

        if response.status_code != 200:
            print(f"{page}페이지 요청 실패")
            continue

        soup = BeautifulSoup(response.text, "html.parser")
        for a in soup.select("dt > a"):
            href = a.get("href")
            if href and href.startswith("https://n.news.naver.com"):
                all_links.add(href)

    print(f"총 {len(all_links)}개의 중복 없는 링크를 수집했습니다.")
    return list(all_links)

# 실행
urls = get_naver_economy_news_urls_from_list(pages=5)


1페이지 응답 코드: 200
2페이지 응답 코드: 200
3페이지 응답 코드: 200
4페이지 응답 코드: 200
5페이지 응답 코드: 200
총 100개의 중복 없는 링크를 수집했습니다.


## Word crawling

In [2]:
def get_news_text(url):
    """
    네이버 뉴스 기사 URL로부터 본문 텍스트를 크롤링하는 함수
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    # 🔹 HTTP GET 요청
    response = requests.get(url, headers=headers)
  
    # 🔸 응답 상태 코드 확인
    if response.status_code != 200:
        print("❌ 뉴스 기사 크롤링 실패:", response.status_code)
        return ""

    # 🔹 HTML 파싱
    soup = BeautifulSoup(response.text, "html.parser")
    
    # 🔸 네이버 뉴스 본문이 있는 div 선택 (2023년 이후 'newsct_article' 사용)
    article_body = soup.select_one("div#newsct_article")

    if article_body:
        # 🔹 줄바꿈 포함하여 텍스트 추출
        return article_body.get_text(strip=True, separator="\n")
    else:
        print("❌ 뉴스 본문을 찾을 수 없음")
        return ""

## Remove particles

In [ ]:
def remove_particles(word):
    """
    한글 단어에서 조사(조사 목록에 포함된 단어)를 제거하는 함수
    예) "전산장애로" → "전산장애"
    """
    # 정규표현식을 통해 단어 끝에 붙은 조사 제거
    pattern = r"(" + "|".join(PARTICLES) + r")$"
    return re.sub(pattern, "", word)



def     (text):
    """
    본문에서 명사를 추출하고 조사 제거 후
    중복 없이 처리하지 않고 등장 순서 그대로 반환
    """
    okt = Okt()
    nouns = okt.phrases(text)
    filtered = [remove_particles(n) for n in nouns if n.strip()]
    return filtered  # 중복 제거 없이 그대로 반환



## Print Word List

In [5]:
def print_words_in_rows(words, words_per_row=10):
    """
    단어 리스트를 지정된 개수(words_per_row)만큼 한 줄에 출력
    """
    for i in range(0, len(words), words_per_row):
        print(", ".join(words[i : i + words_per_row]))

## DB Scanning

In [6]:
# ✅ DB 조회 함수
def find_description_from_db(conn, term_input):
    cursor = conn.cursor()
    query = "SELECT * FROM dictionary WHERE term = ?"
    cursor.execute(query, (term_input,))
    result = cursor.fetchone()
    if result:
        return result[3]  # description 컬럼
    else:
        return None

    # ✅ 메인 실행 함수

def extract_and_explain(url, db_path):
    text = get_news_text(url)
    
    words_Konpy = extract_words_okt(text)
    words_Konpy = list(dict.fromkeys(words_Konpy))  # 중복 제거 (순서 유지)

    conn = sqlite3.connect(db_path)
    found = False  # ✅ 초기화

    print("🔍 기사에서 등장한 경제 용어 설명 Konlpy:\n")
    for word in words_Konpy:
        desc = find_description_from_db(conn, word)
        if desc:
            found = True
            print(f"📌 {word}: {desc}\n")       

    conn.close()

    if not found:
        print("📝 기사 내에서 설명 가능한 경제 용어를 찾지 못했습니다.")

## 크롤링 & 단어 matching test

In [8]:
# ✅ 실행
if __name__ == "__main__":
    db_file = r"C:\Users\wjdgh\OneDrive\바탕 화면\2025-1\2025졸업작품\code\dictionary_test.db"
    
    # 1. 뉴스 URL 수집
    news_urls = get_naver_economy_news_urls_from_list(5)
    
    # 2. 각 뉴스 URL에 대해 처리
    for url in news_urls:
        extract_and_explain(url, db_file)

1페이지 응답 코드: 200
2페이지 응답 코드: 200
3페이지 응답 코드: 200
4페이지 응답 코드: 200
5페이지 응답 코드: 200
총 100개의 중복 없는 링크를 수집했습니다.
🔍 기사에서 등장한 경제 용어 설명 Konlpy:

📌 시가총액: 상장 주식을 시가로 평가한 주식시장에서 평가되는 그 주식의 가치이다. 전체 주식시장의 시가총액은 그 주식시장의 규모를 나타내며, 한 나라 경제크기의 측정치로서 경제지표로 이용할 수 있다. 즉, 국민총생산(GNP)과 비교할 때 국민경제 전체에서 차지하는 주식시장의 비중을 알 수 있고, 경제성장률과 시가총액 증감률 비교를 통해서는 주식시장 성장이 경제성장에 얼마나 선행하는가를 파악할 수 있다.

📌 환율: 화폐간 교환비율. 두 화폐의 상대적 가치를 나타낸다. 만약 미화 1달러를 얻기 위하여 1,000원을 지불한다면, 1달러당 1,000원이 우리 나라 원화의 대미달러 환율이 된다. 화폐가치가 하락하는 것을 (평가) 절하(depreciation), 화폐가치가 상승하는 것을 (평가) 절상(appreciation)이라고 한다.

🔍 기사에서 등장한 경제 용어 설명 Konlpy:

📌 디지털화폐: 디지털 방식으로 사용하는 형태의 화폐로, 금전적 가치를 전자적 형태로 저장해 거래할 수 있는 통화를 말한다. 디지털 화폐를 발행하는 목적은 화폐 발행에 따른 비용을 줄이고, 개인&middot;기업 등 경제주체들의 상품거래 지불수단의 편의성을 높이기 위해서다. 디지털화폐에는 전자화폐, 암호화폐, 중앙은행 디지털화폐(CBDC&middot;Central Bank Digital Currency) 등이 포함된다. 하지만 구조적인 측면에서 볼 때, 가상화폐와 디지털화폐는 이들을 관리하는 중앙기관이 있는 반면 암호화폐는 탈중앙화된 블록체인상에서 이뤄진다는 점에서 이들 모두를 디지털화폐 항목에 포함시킬 수 없다는 의견도 있다.

📌 배드뱅크: 배드뱅크(bad bank)는 금융기관의 부실채권이나 부실자산만을 사들여 이를 전문적으로 처리하는 기관